In [1]:
import pandas as pd
import numpy as np
import time
import math

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
vic_data = pd.read_csv('../../../data/victoria.csv', parse_dates=[0])

In [3]:
BATTERY_POWER = 300
BATTERY_CAPACITY = 580
EFFICIENCY = 0.9
MARGINAL_LOSS_FACTOR = 0.991
FIXED_OPERATIONS_MAINTENANCE = 8.1

TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

In [4]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [5]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    
    df['charge_forecast'] = 0
    df['discharge_forecast'] = 0
    
    df = df.drop([0], axis=0) 
    
    return df

In [6]:
def algorithm3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)


            if (df.at[i,PRICE] <= quantile_exc(ser,CHARGING_PERCENTILE)):
                df.at[i,'charge_forecast'] = 1

            if (df.at[i,PRICE] >= quantile_exc(ser,DISCHARGING_PERCENTILE)):
                df.at[i,'discharge_forecast'] = 1
                
        else:
            thelist = df.iloc[i:][PRICE]
            ser = pd.Series(thelist)

            if (df.at[i,PRICE] <= ser.quantile(CHARGING_PERCENTILE)):
                df.at[i,'charge_forecast'] = 1

            if (df.at[i,PRICE] >= ser.quantile(DISCHARGING_PERCENTILE)):
                df.at[i,'discharge_forecast'] = 1

    return df

In [7]:
POWER = 300
CAPACITY = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

def get_opencap(i, df):
    """Get the opening battery capacity for every 30-minutes interval 
    Input:
        i : Current row in df
        df : DataFrame with 'opening' and 'closing' column
    Return 
        opening_cap : Opening battery capacity. Integer dtype
    
    """
    if i != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = math.ceil(df.at[i, "opening"])
    
    return opening_cap



def get_dispatch(rawPower):
    """ Get the power dispatched for every 30-minutes interval 
    Input:
        rawPower : Current rawPower. Integer Datatype
    Return:
        dispatch : Raw_power dispatched from the market. Integer dtype
    """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = round(rawPower / 2 * eff, 0)
    return dispatch



def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity for every 30-minutes interval
    Input:
        opening_cap : Opening battery capacity. Integer dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        closecap : Closing battery capacity. Integer dtype
    """
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))
    return closecap
    
    
    
def get_revenue(price, dispatch):
    """ Get the Revenue for every 30-minutes interval
    Input:
        price : Market spot price for electricity. Float dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        revenue : current revenue. Integer dtype
    """
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = round(price * dispatch * factor)
    return revenue



def run_algo3(i, df, opening_cap):
    """ Get the Raw Power for every 30-minutes interval
    Input:
        i : Current row in df
        df : DataFrame with 'charge_forecaset' and 'discharge_forecast' column
    Return:
        opening_cap : Opening battery capacity. Integer dtype
    """
    raw_power = 0
    if(df.at[i,'charge_forecast'] == 1):
        raw_power = -min(BATTERY_POWER, (BATTERY_CAPACITY-opening_cap)/EFFICIENCY*2)
    if(df.at[i,'discharge_forecast'] == 1):
        raw_power = min(BATTERY_POWER, opening_cap/EFFICIENCY*2)
            
    return raw_power



def calculate(df):
    """ Calculate the Battery Opening, Closing Capacity, Raw Power, Market dispatch 
        and Revenue for the entire df.
    Input:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    Return :
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    """
    # Go through each 30-minute interval of df
    for i in list(df.index):
        # get current Spot Price
        price = df.at[i, PRICE]
        
        # update opening capacity
        opening_cap = get_opencap(i, df)

        # find raw_power
        rawPower = run_algo3(i, df, opening_cap)
        df.at[i,'raw_power'] = rawPower

        # find market_dispatch 
        dispatch = get_dispatch(rawPower)
        df.at[i,"dispatch"] = dispatch

        # find closing_capacity   
        df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

        #find revenue        
        df.at[i,"revenue"] = get_revenue(price, dispatch)
    
    return df



def show_result(df):
    """ Print the revenue related information computed from df.
    Input: 
        df : DataFrame with 'revenue' column.
    """
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    
    return None

In [8]:
def run_all(ori_df):
    """ Run the entire data pipeline including initialisng (Data Processing), 
    finding the optimal charging and discarging period (Data Modelling), 
    calculating the revenue based on the optimal period mentioned above (Model Testing and Evaluation).
    
    Input:
        ori_df : DataFrame which contains spot_price for every 30-minute interval. 
    Return:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column.
    """
    # Start time
    start = time.time()
    # Initialise df
    df2 = create_df(ori_df)
    # Find Optimal Charging and Discharging period
    df3 = algorithm3(df2)
    # Calculate the revenue
    df = calculate(df3)
    # Show the revenue       
    show_result(df)
    # End Time      
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [9]:
# Highest Revenue using cross validation
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.32
DISCHARGING_PERCENTILE = 0.74
vic_price = run_all(vic_data)

Total revenue in the dataset: 122339837
Total days in the dataset: 1322.0
Revenue per day: 92541.48033282904
Time Complexity for running the entire Algorithm 3: 25.72959280014038s


In [10]:
vic_price.loc[(vic_price['Time (UTC+10)'] >= '2018-01-28') & (vic_price['Time (UTC+10)'] < '2018-01-29')]

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,charge_forecast,discharge_forecast
1296,2018-01-28 00:00:00,84.04,0,0,0,0,0,0,1
1297,2018-01-28 00:30:00,79.86,0,0,0,0,0,0,1
1298,2018-01-28 01:00:00,70.01,0,0,0,0,0,0,1
1299,2018-01-28 01:30:00,64.20,0,0,0,0,0,0,1
1300,2018-01-28 02:00:00,70.39,0,0,0,0,0,0,1
1301,2018-01-28 02:30:00,61.22,0,0,0,0,0,0,1
1302,2018-01-28 03:00:00,59.97,0,0,0,0,0,0,0
1303,2018-01-28 03:30:00,59.72,0,0,0,0,0,0,0
1304,2018-01-28 04:00:00,59.90,0,0,0,0,0,0,0
1305,2018-01-28 04:30:00,61.63,0,0,0,0,0,0,0


In [11]:
thetime = vic_price.loc[(vic_price['Time (UTC+10)'] >= '2020-07-17') & (vic_price['Time (UTC+10)'] < '2020-07-18')]
thetime

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,charge_forecast,discharge_forecast
44544,2020-07-17 00:00:00,79.09,0,0,0,0,0,0,1
44545,2020-07-17 00:30:00,75.51,0,0,0,0,0,0,1
44546,2020-07-17 01:00:00,73.98,0,0,0,0,0,0,0
44547,2020-07-17 01:30:00,75.57,0,0,0,0,0,0,1
44548,2020-07-17 02:00:00,71.94,0,0,0,0,0,0,0
44549,2020-07-17 02:30:00,74.10,0,0,0,0,0,0,0
44550,2020-07-17 03:00:00,67.36,0,0,0,0,0,0,0
44551,2020-07-17 03:30:00,58.04,-300,-150,-8785,0,135,1,0
44552,2020-07-17 04:00:00,51.85,-300,-150,-7848,135,270,1,0
44553,2020-07-17 04:30:00,74.53,0,0,0,270,270,0,0


In [12]:
thetime.to_csv("../../data/third_deliverable.csv")

In [26]:
one_cycle = thetime.loc[(thetime['Time (UTC+10)'] >= '2020-07-17 03:30:00') & (thetime['Time (UTC+10)'] < '2020-07-17 11:30:00')]
one_cycle

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,charge_forecast,discharge_forecast
44551,2020-07-17 03:30:00,58.04,-300,-150,-8785,0,135,1,0
44552,2020-07-17 04:00:00,51.85,-300,-150,-7848,135,270,1,0
44553,2020-07-17 04:30:00,74.53,0,0,0,270,270,0,0
44554,2020-07-17 05:00:00,67.32,-66,-33,-2242,270,300,1,0
44555,2020-07-17 05:30:00,69.14,0,0,0,300,300,1,0
44556,2020-07-17 06:00:00,64.99,0,0,0,300,300,1,0
44557,2020-07-17 06:30:00,65.35,0,0,0,300,300,1,0
44558,2020-07-17 07:00:00,76.16,0,0,0,300,300,0,0
44559,2020-07-17 07:30:00,79.41,0,0,0,300,300,0,0
44560,2020-07-17 08:00:00,83.26,0,0,0,300,300,0,0


In [27]:
sum(one_cycle['revenue'])

14235

In [28]:
sum(thetime['revenue'])

41054

In [14]:
# Highest Revenue so far
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.3
DISCHARGING_PERCENTILE = 0.75
vic_price = run_all(vic_data)

Total revenue in the dataset: 122130345
Total days in the dataset: 1322.0
Revenue per day: 92383.0143721634
Time Complexity for running the entire Algorithm 3: 23.732851028442383s


In [12]:
# second most optimal solution!
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.25
DISCHARGING_PERCENTILE = 0.75
vic_price = run_all(vic_data)

Total revenue in the dataset: 121619931
Total days in the dataset: 1322.0
Revenue per day: 91996.92208774584
Time Complexity for running the entire Algorithm 3: 21.236649990081787s


In [15]:
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.15
DISCHARGING_PERCENTILE = 0.85
vic_price = run_all(vic_data)

Total revenue in the dataset: 112444396
Total days in the dataset: 1322.0
Revenue per day: 85056.27534039335
Time Complexity for running the entire Algorithm 3: 21.27220916748047s


In [16]:
LOOKAHEAD = 6
CHARGING_PERCENTILE = 0.15
DISCHARGING_PERCENTILE = 0.85
vic_price = run_all(vic_data)

Total revenue in the dataset: 101334370
Total days in the dataset: 1322.0
Revenue per day: 76652.32223903177
Time Complexity for running the entire Algorithm 3: 23.34444808959961s


In [27]:
# third most optimal solution!
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.3
DISCHARGING_PERCENTILE = 0.7
vic_price = run_all(vic_data)

Total revenue in the dataset: 121520226
Total days in the dataset: 1322.0
Revenue per day: 91921.50226928896
Time Complexity for running the entire Algorithm 3: 22.29344415664673s


In [18]:
LOOKAHEAD = 6
CHARGING_PERCENTILE = 0.35
DISCHARGING_PERCENTILE = 0.65
vic_price = run_all(vic_data)

Total revenue in the dataset: 118929786
Total days in the dataset: 1322.0
Revenue per day: 89962.01664145234
Time Complexity for running the entire Algorithm 3: 20.557847023010254s


In [19]:
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.35
DISCHARGING_PERCENTILE = 0.65
vic_price = run_all(vic_data)

Total revenue in the dataset: 120842805
Total days in the dataset: 1322.0
Revenue per day: 91409.08093797277
Time Complexity for running the entire Algorithm 3: 21.502471923828125s


In [24]:
LOOKAHEAD = 15
CHARGING_PERCENTILE = 0.2
DISCHARGING_PERCENTILE = 0.8
vic_price = run_all(vic_data)

Total revenue in the dataset: 117624254
Total days in the dataset: 1322.0
Revenue per day: 88974.47352496217
Time Complexity for running the entire Algorithm 3: 22.940332889556885s


In [25]:
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.2
DISCHARGING_PERCENTILE = 0.8
vic_price = run_all(vic_data)

Total revenue in the dataset: 117366675
Total days in the dataset: 1322.0
Revenue per day: 88779.63313161876
Time Complexity for running the entire Algorithm 3: 22.873759031295776s


In [26]:
LOOKAHEAD = 6
CHARGING_PERCENTILE = 0.2
DISCHARGING_PERCENTILE = 0.8
vic_price = run_all(vic_data)

Total revenue in the dataset: 109875397
Total days in the dataset: 1322.0
Revenue per day: 83113.00832072618
Time Complexity for running the entire Algorithm 3: 21.245874881744385s
